# 📊 Microsoft Fabric – Notebook & Pipeline Efficiency Monitor

This notebook discovers all **Notebooks** and **Data Pipelines** in a Microsoft Fabric workspace and analyzes their **execution efficiency** using the Fabric REST API.

| Metric | Source |
|--------|--------|
| Item Name & Type | Fabric REST API – Items |
| Run History (Start, End, Status) | Fabric REST API – Job Instances |
| Duration (seconds / minutes) | Calculated from job start & end times |
| Success / Failure Rate | Calculated from job status |
| Average / P50 / P95 Duration | Statistical analysis over run history |
| Spark Resource Efficiency % | Fabric Spark Applications API + job instance analysis |
| Capacity Unit (CU) Seconds | Fabric REST API – Job Details (when available) |
| Error Messages | Fabric REST API – Failed job details |

```mermaid
flowchart LR
    A["🔐 Fabric Auth"] --> B["📡 Fabric REST API"]
    B --> C["📓 List Notebooks"]
    B --> D["🔄 List Pipelines"]
    C --> E["📜 Job Run History"]
    D --> E
    E --> F["📊 Efficiency Metrics"]
    E --> S["⚡ Spark Resource Efficiency"]
    F --> G["📋 Consolidated Report"]
    S --> G
    G --> H["💾 CSV Export"]
```

## Prerequisites
- This notebook runs **inside Microsoft Fabric** (not externally)
- Fabric workspace Admin or Member role
- Pre-installed libraries: `sempy`, `mssparkutils`, `pandas`, `requests`

## 0️⃣ Verify Fabric Environment

In [ ]:
# ── Verify Fabric Environment ─────────────────────────────────────────

import sempy.fabric as fabric
from notebookutils import mssparkutils
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, timezone
import time

print("✅ Fabric environment verified")
print(f"   sempy version:      {fabric.__version__ if hasattr(fabric, '__version__') else 'OK'}")
print(f"   mssparkutils:       available")
print(f"   pandas version:     {pd.__version__}")

## 1️⃣ Configuration

Set your Fabric workspace ID and analysis parameters below.
You can find your **Workspace ID** in the Fabric portal URL:
`https://app.fabric.microsoft.com/groups/<WORKSPACE_ID>/...`

In [ ]:
# ── Configuration ────────────────────────────────────────────────────

# ── Fabric Workspace ───────────────────────────────────────────────
# Set to None to use the current workspace, or specify a workspace GUID
TARGET_WORKSPACE_ID = None

# ── Analysis Window ────────────────────────────────────────────────
# Number of days of run history to analyze (max depends on Fabric retention)
LOOKBACK_DAYS = 30

# ── Item Filters ────────────────────────────────────────────────────
# Set to True/False to include/exclude specific item types
INCLUDE_NOTEBOOKS = True
INCLUDE_PIPELINES = True

# ── Resolve workspace ID ────────────────────────────────────────────
if TARGET_WORKSPACE_ID:
    WORKSPACE_ID = TARGET_WORKSPACE_ID
else:
    WORKSPACE_ID = fabric.get_workspace_id()

# ── Fabric API ─────────────────────────────────────────────────────
FABRIC_API_BASE = "https://api.fabric.microsoft.com/v1"

# ── Authentication via mssparkutils ──────────────────────────────────
fabric_token = mssparkutils.credentials.getToken("pbi")

HEADERS = {
    "Authorization": f"Bearer {fabric_token}",
    "Content-Type": "application/json"
}

# ── Display Options ────────────────────────────────────────────────
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.float_format', '{:,.2f}'.format)

# ── Validate – fetch workspace info ──────────────────────────────────
resp = requests.get(f"{FABRIC_API_BASE}/workspaces/{WORKSPACE_ID}", headers=HEADERS)
resp.raise_for_status()
ws = resp.json()
print(f"📊 Notebook & Pipeline Efficiency Monitor")
print(f"   Workspace:      {ws['displayName']}")
print(f"   Workspace ID:   {WORKSPACE_ID}")
print(f"   Capacity:       {ws.get('capacityId', 'N/A')}")
print(f"   Lookback:       {LOOKBACK_DAYS} days")
print(f"   Include:        {'Notebooks' if INCLUDE_NOTEBOOKS else ''} {'Pipelines' if INCLUDE_PIPELINES else ''}")

In [ ]:
# ── Discover Notebooks & Data Pipelines ─────────────────────────────

def list_fabric_items_by_type(workspace_id, headers, item_type=None):
    """
    List items in a Fabric workspace. Optionally filter by type.
    Handles pagination via continuationToken.
    """
    url = f"{FABRIC_API_BASE}/workspaces/{workspace_id}/items"
    if item_type:
        url += f"?type={item_type}"

    items = []
    while url:
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()
        items.extend(data.get("value", []))
        # Handle pagination
        continuation_token = data.get("continuationToken")
        if continuation_token:
            separator = "&" if "?" in url.split("continuationToken")[0] else "?"
            base_url = url.split("&continuationToken")[0].split("?continuationToken")[0]
            url = f"{base_url}{separator}continuationToken={continuation_token}"
        else:
            url = None
    return items


# ── Notebooks ──────────────────────────────────────────────────────
notebooks = []
if INCLUDE_NOTEBOOKS:
    raw_notebooks = list_fabric_items_by_type(WORKSPACE_ID, HEADERS, "Notebook")
    for nb in raw_notebooks:
        notebooks.append({
            "item_id":   nb["id"],
            "item_name": nb["displayName"],
            "item_type": "Notebook",
            "description": nb.get("description", ""),
        })
    print(f"📓 Found {len(notebooks)} Notebook(s)")

# ── Data Pipelines ─────────────────────────────────────────────────
pipelines = []
if INCLUDE_PIPELINES:
    raw_pipelines = list_fabric_items_by_type(WORKSPACE_ID, HEADERS, "DataPipeline")
    for dp in raw_pipelines:
        pipelines.append({
            "item_id":   dp["id"],
            "item_name": dp["displayName"],
            "item_type": "DataPipeline",
            "description": dp.get("description", ""),
        })
    print(f"🔄 Found {len(pipelines)} Data Pipeline(s)")

# ── Combined list ───────────────────────────────────────────────────
all_items = notebooks + pipelines
df_items = pd.DataFrame(all_items)

print(f"\n✅ Total: {len(all_items)} item(s) to analyze")
for item in all_items:
    print(f"   {item['item_type']:14s} | {item['item_name']}")

## 3️⃣ Fetch Run History (Job Instances)

Queries the Fabric REST API for **job instances** (run history) for each Notebook and Pipeline.
This captures start time, end time, status, and failure details for every execution.

> **Note:** The Fabric API returns job instances based on retention policies of your capacity.
> Adjust `LOOKBACK_DAYS` in the Configuration cell to control the analysis window.

In [ ]:
# ── Fetch Run History ────────────────────────────────────────────────

def get_job_instances(workspace_id, item_id, headers, lookback_days=30):
    """
    Retrieve job instances (run history) for a specific Fabric item.
    Uses GET /v1/workspaces/{workspaceId}/items/{itemId}/jobs/instances
    """
    url = (f"{FABRIC_API_BASE}/workspaces/{workspace_id}"
           f"/items/{item_id}/jobs/instances")

    all_instances = []
    cutoff = datetime.now(timezone.utc) - timedelta(days=lookback_days)

    while url:
        try:
            resp = requests.get(url, headers=headers)
            if resp.status_code == 404:
                # Item may not support job instances
                return []
            resp.raise_for_status()
            data = resp.json()
            instances = data.get("value", [])
            all_instances.extend(instances)

            # Handle pagination
            continuation_token = data.get("continuationToken")
            if continuation_token:
                base_url = url.split("?continuationToken")[0].split("&continuationToken")[0]
                separator = "&" if "?" in base_url else "?"
                url = f"{base_url}{separator}continuationToken={continuation_token}"
            else:
                url = None
        except requests.exceptions.HTTPError as e:
            if e.response.status_code in (400, 403, 404):
                return []
            raise

    return all_instances


def parse_datetime(dt_str):
    """Parse ISO datetime string, handling various formats. Always returns timezone-aware (UTC)."""
    if not dt_str:
        return None
    try:
        # Handle 'Z' suffix and microseconds
        dt_str = dt_str.replace("Z", "+00:00")
        dt = datetime.fromisoformat(dt_str)
        # Ensure timezone-aware – if naive, assume UTC
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt
    except (ValueError, TypeError):
        return None


# ── Collect run history for all items ────────────────────────────────
all_runs = []
items_with_runs = 0
items_without_runs = 0

for item in all_items:
    item_id   = item["item_id"]
    item_name = item["item_name"]
    item_type = item["item_type"]

    print(f"📜 Fetching run history for {item_type}: {item_name} ...")

    instances = get_job_instances(WORKSPACE_ID, item_id, HEADERS, LOOKBACK_DAYS)

    if instances:
        items_with_runs += 1
        for inst in instances:
            start_time = parse_datetime(inst.get("startTimeUtc"))
            end_time = parse_datetime(inst.get("endTimeUtc"))

            # Calculate duration
            duration_seconds = None
            if start_time and end_time:
                duration_seconds = (end_time - start_time).total_seconds()

            # Filter by lookback window
            cutoff = datetime.now(timezone.utc) - timedelta(days=LOOKBACK_DAYS)
            if start_time and start_time < cutoff:
                continue

            all_runs.append({
                "item_id":          item_id,
                "item_name":        item_name,
                "item_type":        item_type,
                "job_instance_id":  inst.get("id", ""),
                "job_type":         inst.get("jobType", "Unknown"),
                "invoke_type":      inst.get("invokeType", "Unknown"),
                "status":           inst.get("status", "Unknown"),
                "start_time":       start_time,
                "end_time":         end_time,
                "duration_seconds": duration_seconds,
                "failure_reason":   inst.get("failureReason", {}).get("message", "")
                                    if isinstance(inst.get("failureReason"), dict)
                                    else str(inst.get("failureReason", "")),
                "root_activity_id": inst.get("rootActivityId", ""),
            })

        print(f"   ✅ {len([r for r in all_runs if r['item_id'] == item_id])} run(s) within lookback window")
    else:
        items_without_runs += 1
        print(f"   ℹ️  No run history found")

    # Rate limiting – small delay between API calls
    time.sleep(0.2)

df_runs = pd.DataFrame(all_runs)

print(f"\n📊 Run History Summary")
print(f"   Items with runs:    {items_with_runs}")
print(f"   Items without runs: {items_without_runs}")
print(f"   Total runs found:   {len(all_runs)}")

## 4️⃣ Calculate Efficiency Metrics

Computes per-item efficiency metrics from the run history:

| Metric | Description |
|--------|-------------|
| **Total Runs** | Number of executions in the lookback window |
| **Success / Failure Count** | Runs by status |
| **Success Rate (%)** | Percentage of successful runs |
| **Avg Duration (min)** | Mean execution time |
| **P50 Duration (min)** | Median execution time |
| **P95 Duration (min)** | 95th percentile execution time |
| **Min / Max Duration (min)** | Shortest and longest execution |
| **Total Compute Time (hrs)** | Cumulative execution time |
| **Last Run** | Most recent execution timestamp |
| **Last Status** | Status of the most recent run |
| **Trend** | Duration trend (improving / degrading / stable) |

In [ ]:
# ── Calculate Efficiency Metrics ─────────────────────────────────────

import numpy as np

def calculate_efficiency_metrics(df_runs):
    """
    Calculate per-item efficiency metrics from run history.
    Returns a DataFrame with one row per item.
    """
    if df_runs.empty:
        return pd.DataFrame()

    metrics = []

    for (item_id, item_name, item_type), group in df_runs.groupby(
        ["item_id", "item_name", "item_type"]
    ):
        total_runs = len(group)
        successful = group[group["status"].str.lower().isin(["completed", "succeeded"])]
        failed = group[group["status"].str.lower().isin(["failed", "cancelled", "deduped"])]
        in_progress = group[group["status"].str.lower().isin(["inprogress", "running", "notstarted"])]

        success_count = len(successful)
        failure_count = len(failed)
        in_progress_count = len(in_progress)
        success_rate = (success_count / total_runs * 100) if total_runs > 0 else 0

        # Duration stats (only for completed runs with valid duration)
        durations = group["duration_seconds"].dropna()
        durations = durations[durations >= 0]

        avg_duration_sec = durations.mean() if len(durations) > 0 else None
        p50_duration_sec = durations.median() if len(durations) > 0 else None
        p95_duration_sec = durations.quantile(0.95) if len(durations) > 0 else None
        min_duration_sec = durations.min() if len(durations) > 0 else None
        max_duration_sec = durations.max() if len(durations) > 0 else None
        total_compute_sec = durations.sum() if len(durations) > 0 else 0

        # Last run info
        sorted_runs = group.sort_values("start_time", ascending=False)
        last_run_time = sorted_runs["start_time"].iloc[0] if not sorted_runs.empty else None
        last_status = sorted_runs["status"].iloc[0] if not sorted_runs.empty else "N/A"

        # Duration trend (compare first half vs second half of runs)
        trend = "N/A"
        if len(durations) >= 4:
            sorted_group = group.dropna(subset=["start_time", "duration_seconds"]).sort_values("start_time")
            if len(sorted_group) >= 4:
                mid = len(sorted_group) // 2
                first_half_avg = sorted_group.iloc[:mid]["duration_seconds"].mean()
                second_half_avg = sorted_group.iloc[mid:]["duration_seconds"].mean()
                if first_half_avg > 0:
                    change_pct = ((second_half_avg - first_half_avg) / first_half_avg) * 100
                    if change_pct < -10:
                        trend = f"⬆️ Improving ({change_pct:+.0f}%)"
                    elif change_pct > 10:
                        trend = f"⬇️ Degrading ({change_pct:+.0f}%)"
                    else:
                        trend = f"➡️ Stable ({change_pct:+.0f}%)"

        # Most common failure reason
        failure_reasons = group[group["failure_reason"].str.len() > 0]["failure_reason"]
        top_failure = failure_reasons.mode().iloc[0] if len(failure_reasons) > 0 else ""

        # Invoke type distribution
        invoke_dist = group["invoke_type"].value_counts().to_dict()

        metrics.append({
            "item_id":              item_id,
            "item_name":            item_name,
            "item_type":            item_type,
            "total_runs":           total_runs,
            "success_count":        success_count,
            "failure_count":        failure_count,
            "in_progress_count":    in_progress_count,
            "success_rate_pct":     round(success_rate, 1),
            "avg_duration_min":     round(avg_duration_sec / 60, 2) if avg_duration_sec else None,
            "p50_duration_min":     round(p50_duration_sec / 60, 2) if p50_duration_sec else None,
            "p95_duration_min":     round(p95_duration_sec / 60, 2) if p95_duration_sec else None,
            "min_duration_min":     round(min_duration_sec / 60, 2) if min_duration_sec else None,
            "max_duration_min":     round(max_duration_sec / 60, 2) if max_duration_sec else None,
            "total_compute_hrs":    round(total_compute_sec / 3600, 2) if total_compute_sec else 0,
            "last_run_time":        last_run_time,
            "last_status":          last_status,
            "duration_trend":       trend,
            "top_failure_reason":   top_failure[:100] if top_failure else "",
            "invoke_types":         str(invoke_dist),
        })

    return pd.DataFrame(metrics)


if not df_runs.empty:
    df_efficiency = calculate_efficiency_metrics(df_runs)
    print(f"✅ Efficiency metrics calculated for {len(df_efficiency)} item(s)")
else:
    df_efficiency = pd.DataFrame()
    print("⚠️  No run data available to calculate efficiency metrics.")

## 5️⃣ Efficiency Report – All Items

Consolidated view showing efficiency metrics for every Notebook and Pipeline.
Items are ranked by success rate and then by total runs.

In [ ]:
# ── Efficiency Report ────────────────────────────────────────────────

if not df_efficiency.empty:
    # Sort: lowest success rate first (most problematic items on top)
    df_sorted = df_efficiency.sort_values(
        ["success_rate_pct", "total_runs"],
        ascending=[True, False]
    )

    display_cols = [
        "item_type", "item_name", "total_runs",
        "success_count", "failure_count", "success_rate_pct",
        "avg_duration_min", "p50_duration_min", "p95_duration_min",
        "total_compute_hrs", "last_status", "duration_trend"
    ]

    df_display = df_sorted[display_cols].rename(columns={
        "item_type":         "Type",
        "item_name":         "Name",
        "total_runs":        "Runs",
        "success_count":     "✅ Pass",
        "failure_count":     "❌ Fail",
        "success_rate_pct":  "Success %",
        "avg_duration_min":  "Avg (min)",
        "p50_duration_min":  "P50 (min)",
        "p95_duration_min":  "P95 (min)",
        "total_compute_hrs": "Total (hrs)",
        "last_status":       "Last Status",
        "duration_trend":    "Trend",
    })

    print("📋 NOTEBOOK & PIPELINE EFFICIENCY REPORT")
    print("=" * 140)
    print(f"   Workspace:    {ws['displayName']}")
    print(f"   Period:       Last {LOOKBACK_DAYS} days")
    print(f"   Items:        {len(df_efficiency)} ({len(df_efficiency[df_efficiency['item_type']=='Notebook'])} Notebooks, "
          f"{len(df_efficiency[df_efficiency['item_type']=='DataPipeline'])} Pipelines)")
    total_runs = df_efficiency['total_runs'].sum()
    overall_success = (df_efficiency['success_count'].sum() / total_runs * 100) if total_runs > 0 else 0
    print(f"   Total Runs:   {total_runs:,}")
    print(f"   Overall Success Rate: {overall_success:.1f}%")
    total_compute = df_efficiency['total_compute_hrs'].sum()
    print(f"   Total Compute Time:   {total_compute:,.1f} hours")
    print("=" * 140)
    print()
    print(df_display.to_string(index=False))
else:
    print("⚠️  No efficiency data to display. Ensure previous cells ran successfully.")

## 6️⃣ Health Dashboard – At-a-Glance

Color-coded summary showing item health based on success rate thresholds:
- 🟢 **Healthy**: Success rate ≥ 95%
- 🟡 **Warning**: Success rate 80%–94%
- 🔴 **Critical**: Success rate < 80%

In [ ]:
# ── Health Dashboard ─────────────────────────────────────────────────

if not df_efficiency.empty:
    def health_badge(rate):
        if rate >= 95:
            return "🟢 Healthy"
        elif rate >= 80:
            return "🟡 Warning"
        else:
            return "🔴 Critical"

    df_health = df_efficiency.copy()
    df_health["health"] = df_health["success_rate_pct"].apply(health_badge)

    print("🏥 HEALTH DASHBOARD")
    print("=" * 100)

    # Summary counts
    healthy  = len(df_health[df_health["success_rate_pct"] >= 95])
    warning  = len(df_health[(df_health["success_rate_pct"] >= 80) & (df_health["success_rate_pct"] < 95)])
    critical = len(df_health[df_health["success_rate_pct"] < 80])
    never_run = len(df_items) - len(df_efficiency) if not df_items.empty else 0

    print(f"   🟢 Healthy:    {healthy} item(s)")
    print(f"   🟡 Warning:    {warning} item(s)")
    print(f"   🔴 Critical:   {critical} item(s)")
    if never_run > 0:
        print(f"   ⚪ Never Run:  {never_run} item(s)")
    print("=" * 100)

    # Detail by health status
    for status in ["🔴 Critical", "🟡 Warning", "🟢 Healthy"]:
        subset = df_health[df_health["health"] == status].sort_values("success_rate_pct")
        if subset.empty:
            continue
        print(f"\n{status} ({len(subset)} items)")
        print("-" * 100)
        for _, row in subset.iterrows():
            icon = "📓" if row["item_type"] == "Notebook" else "🔄"
            last_run_str = row["last_run_time"].strftime("%Y-%m-%d %H:%M") if pd.notna(row["last_run_time"]) else "N/A"
            print(f"   {icon} {row['item_name']}")
            print(f"      Runs: {row['total_runs']:,}  |  Success: {row['success_rate_pct']}%  "
                  f"|  Avg: {row['avg_duration_min']:.1f} min  |  Last: {last_run_str}  {row['duration_trend']}")
            if row["top_failure_reason"]:
                print(f"      ⚠️ Top Failure: {row['top_failure_reason'][:80]}")
        print()

    # Items that have never been executed
    if not df_items.empty:
        executed_ids = set(df_efficiency["item_id"])
        never_executed = df_items[~df_items["item_id"].isin(executed_ids)]
        if not never_executed.empty:
            print(f"\n⚪ NEVER EXECUTED ({len(never_executed)} items)")
            print("-" * 60)
            for _, row in never_executed.iterrows():
                icon = "📓" if row["item_type"] == "Notebook" else "🔄"
                print(f"   {icon} {row['item_name']} ({row['item_type']})")
else:
    print("⚠️  No data available.")

## 7️⃣ Failure Analysis

Deep dive into failed runs – identifies the most common failure reasons and the items with the highest failure rates.

In [ ]:
# ── Failure Analysis ─────────────────────────────────────────────────

if not df_runs.empty:
    failed_runs = df_runs[df_runs["status"].str.lower().isin(["failed", "cancelled"])]

    if not failed_runs.empty:
        print("❌ FAILURE ANALYSIS")
        print("=" * 120)
        print(f"   Total Failed/Cancelled Runs: {len(failed_runs)}")
        print()

        # ── Top failing items ────────────────────────────────────────
        print("📊 Top Failing Items")
        print("-" * 80)
        fail_by_item = failed_runs.groupby(["item_type", "item_name"]).agg(
            fail_count=("status", "count"),
        ).reset_index().sort_values("fail_count", ascending=False).head(10)

        for _, row in fail_by_item.iterrows():
            icon = "📓" if row["item_type"] == "Notebook" else "🔄"
            print(f"   {icon} {row['item_name']}: {row['fail_count']} failure(s)")

        # ── Most common failure reasons ────────────────────────────────
        print("\n📋 Most Common Failure Reasons")
        print("-" * 80)
        reasons = failed_runs[failed_runs["failure_reason"].str.len() > 0]["failure_reason"]
        if not reasons.empty:
            top_reasons = reasons.value_counts().head(10)
            for reason, count in top_reasons.items():
                truncated = reason[:100] + "..." if len(reason) > 100 else reason
                print(f"   [{count}x] {truncated}")
        else:
            print("   No failure reasons recorded in the API response.")

        # ── Recent failures timeline ──────────────────────────────────
        print("\n📅 Recent Failures (Last 10)")
        print("-" * 120)
        recent_failures = failed_runs.dropna(subset=["start_time"]).sort_values(
            "start_time", ascending=False
        ).head(10)

        for _, row in recent_failures.iterrows():
            start_str = row["start_time"].strftime("%Y-%m-%d %H:%M") if pd.notna(row["start_time"]) else "N/A"
            dur = f"{row['duration_seconds']/60:.1f} min" if pd.notna(row["duration_seconds"]) else "N/A"
            reason = row["failure_reason"][:60] + "..." if len(str(row["failure_reason"])) > 60 else row["failure_reason"]
            icon = "📓" if row["item_type"] == "Notebook" else "🔄"
            print(f"   {start_str} | {icon} {row['item_name'][:30]:30s} | {dur:>10s} | {reason}")
    else:
        print("✅ No failed runs in the analysis window – all executions succeeded!")
else:
    print("⚠️  No run data available.")

## 8️⃣ Run Duration Trends

Shows how execution duration has changed over time for the most active items.
Helps identify performance degradation or improvement patterns.

In [ ]:
# ── Run Duration Trends ──────────────────────────────────────────────

if not df_runs.empty:
    # Only analyze completed runs with valid duration
    completed = df_runs[
        (df_runs["status"].str.lower().isin(["completed", "succeeded"])) &
        (df_runs["duration_seconds"].notna()) &
        (df_runs["start_time"].notna())
    ].copy()

    if not completed.empty:
        completed["run_date"] = completed["start_time"].dt.date

        # Top 10 most frequently run items
        top_items = completed["item_name"].value_counts().head(10).index.tolist()

        print("📈 DURATION TRENDS (Top 10 Most Active Items)")
        print("=" * 120)

        for item_name in top_items:
            item_runs = completed[completed["item_name"] == item_name].sort_values("start_time")
            item_type = item_runs["item_type"].iloc[0]
            icon = "📓" if item_type == "Notebook" else "🔄"

            print(f"\n{icon} {item_name} ({len(item_runs)} runs)")
            print("-" * 80)

            # Weekly aggregation
            item_runs["week"] = item_runs["start_time"].dt.isocalendar().week
            weekly = item_runs.groupby("week").agg(
                runs=("duration_seconds", "count"),
                avg_dur_min=("duration_seconds", lambda x: x.mean() / 60),
                p95_dur_min=("duration_seconds", lambda x: x.quantile(0.95) / 60),
            ).reset_index()

            for _, row in weekly.iterrows():
                bar_len = int(row["avg_dur_min"] * 2)  # Scale bar
                bar = "█" * min(bar_len, 50)
                print(f"   Week {int(row['week']):2d} | "
                      f"Runs: {int(row['runs']):3d} | "
                      f"Avg: {row['avg_dur_min']:7.1f} min | "
                      f"P95: {row['p95_dur_min']:7.1f} min | {bar}")
    else:
        print("ℹ️  No completed runs with duration data found.")
else:
    print("⚠️  No run data available.")

## 9️⃣ Scheduling & Trigger Analysis

Analyzes how Notebooks and Pipelines are invoked — manually, on-schedule, or via other triggers.
Identifies items that may benefit from scheduling or have irregular execution patterns.

In [ ]:
# ── Scheduling & Trigger Analysis ────────────────────────────────────

if not df_runs.empty:
    print("🕐 SCHEDULING & TRIGGER ANALYSIS")
    print("=" * 100)

    # Invoke type distribution
    invoke_summary = df_runs.groupby(["item_type", "item_name", "invoke_type"]).size().reset_index(name="count")
    invoke_pivot = invoke_summary.pivot_table(
        index=["item_type", "item_name"],
        columns="invoke_type",
        values="count",
        fill_value=0
    ).reset_index()

    print("\n📊 Invocation Method Distribution")
    print("-" * 100)
    print(invoke_pivot.to_string(index=False))

    # Execution frequency analysis
    print("\n\n📅 Execution Frequency")
    print("-" * 100)

    for (item_id, item_name, item_type), group in df_runs.groupby(
        ["item_id", "item_name", "item_type"]
    ):
        if len(group) < 2:
            continue

        sorted_runs = group.dropna(subset=["start_time"]).sort_values("start_time")
        if len(sorted_runs) < 2:
            continue

        time_diffs = sorted_runs["start_time"].diff().dropna()
        avg_interval = time_diffs.mean()
        icon = "📓" if item_type == "Notebook" else "🔄"

        # Determine pattern
        if avg_interval <= timedelta(hours=1):
            frequency = f"~{avg_interval.total_seconds()/60:.0f} min (sub-hourly)"
        elif avg_interval <= timedelta(hours=24):
            frequency = f"~{avg_interval.total_seconds()/3600:.1f} hrs (daily pattern)"
        elif avg_interval <= timedelta(days=7):
            frequency = f"~{avg_interval.days} day(s) (weekly pattern)"
        else:
            frequency = f"~{avg_interval.days} day(s) (infrequent)"

        print(f"   {icon} {item_name}: {frequency}  ({len(sorted_runs)} runs)")

else:
    print("⚠️  No run data available.")

## 9️⃣b Spark Resource Efficiency (%)

Tracks the **Spark Resource Efficiency %** for every notebook in the workspace — the same metric
shown in the Fabric portal under each notebook run → **Resources** tab → **Spark resource usage**.

| Metric | Description |
|--------|-------------|
| **Spark Efficiency %** | `Active Executor-Core-Seconds / Allocated Executor-Core-Seconds × 100` |
| **Total Duration** | Wall-clock time the Spark application ran |
| **Total Idle Time** | Time allocated cores were idle (not processing tasks) |
| **Compute Efficiency** | Successful compute time / Total compute time (from job instances) |
| **Duration Consistency** | `P50 / Mean × 100` – higher = less variance across runs |

This section tries the **Fabric Spark Applications API** first for real core-level efficiency.
If the API is not accessible, it computes a **Compute Efficiency** proxy from job instance data.

> **Note:** The real Spark core-utilization efficiency (shown in the portal screenshot) requires
> access to the Spark monitoring API. If unavailable, you can view per-run efficiency in the
> Fabric portal: open a notebook run → **Resources** tab → **Spark resource usage**.

In [ ]:
# ── Spark Resource Efficiency Tracking ───────────────────────────────

def try_get_spark_applications(workspace_id, headers, lookback_days=30):
    """
    Attempt to retrieve Spark application data from Fabric APIs.
    Tries the Spark Applications API first, then falls back to Livy Sessions.
    Returns (list_of_apps, api_source) or (None, None).
    """
    from_ts = (datetime.now(timezone.utc) - timedelta(days=lookback_days)).strftime("%Y-%m-%dT%H:%M:%SZ")

    # ── Attempt 1: Fabric Spark Applications API ──────────────────
    endpoints = [
        (f"{FABRIC_API_BASE}/workspaces/{workspace_id}/spark/applications",
         {"fromTimestamp": from_ts}, "Spark Applications API"),
        (f"{FABRIC_API_BASE}/workspaces/{workspace_id}/spark/livySessions",
         {}, "Livy Sessions API"),
    ]

    for url, params, source_name in endpoints:
        try:
            resp = requests.get(url, headers=headers, params=params, timeout=30)
            if resp.status_code == 200:
                data = resp.json()
                apps = data.get("value", data.get("sessions", data if isinstance(data, list) else []))
                if apps:
                    return apps, source_name
        except Exception:
            pass

    return None, None


def parse_spark_app_efficiency(apps):
    """
    Parse Spark application data and extract efficiency metrics.
    Maps variable field names across different Spark API versions.
    """
    results = []
    for app in apps:
        name   = app.get("name", app.get("appName", "Unknown"))
        app_id = app.get("id", app.get("appId", app.get("applicationId", "")))
        state  = app.get("state", app.get("status", ""))

        # ── Core efficiency metrics ──────────────────────────────────
        total_seconds = app.get("totalDuration", app.get("duration", None))
        if total_seconds and isinstance(total_seconds, str):
            total_seconds = None  # Must be numeric
        idle_seconds  = app.get("totalIdleTime", app.get("idleTime", None))
        efficiency    = app.get("efficiency", None)

        # Duration from startTime/endTime if totalDuration not available
        if total_seconds is None:
            start_str = app.get("startTime", app.get("startedAt", ""))
            end_str   = app.get("endTime", app.get("finishedAt", ""))
            if start_str and end_str:
                try:
                    st = parse_datetime(str(start_str))
                    et = parse_datetime(str(end_str))
                    if st and et:
                        total_seconds = (et - st).total_seconds()
                except Exception:
                    pass

        # Calculate efficiency from components if not directly available
        if efficiency is None and total_seconds and idle_seconds:
            if total_seconds > 0:
                efficiency = ((total_seconds - idle_seconds) / total_seconds) * 100

        # Executor core info
        total_cores = app.get("totalExecutorCores", app.get("executorCores", None))
        executors   = app.get("executorCount", app.get("numExecutors", None))

        results.append({
            "notebook_name":           name,
            "app_id":                  str(app_id),
            "state":                   state,
            "total_duration_sec":      total_seconds,
            "total_idle_sec":          idle_seconds,
            "spark_efficiency_pct":    round(efficiency, 2) if efficiency is not None else None,
            "executor_cores":          total_cores,
            "executor_count":          executors,
        })

    return pd.DataFrame(results) if results else pd.DataFrame()


def compute_notebook_efficiency_from_jobs(df_runs, notebooks):
    """
    Calculate Compute Efficiency per notebook from job instance data.

    Compute Efficiency = Successful Compute Time / Total Compute Time × 100
    Consistency        = Duration P50 / Mean × 100 (higher = less variance)
    Estimated Efficiency = 70% Compute + 30% Consistency
    """
    if df_runs.empty or not notebooks:
        return pd.DataFrame()

    nb_runs = df_runs[df_runs["item_type"] == "Notebook"].copy()
    if nb_runs.empty:
        return pd.DataFrame()

    results = []
    for nb in notebooks:
        runs = nb_runs[nb_runs["item_id"] == nb["item_id"]]
        if runs.empty:
            continue

        total_runs  = len(runs)
        completed   = runs[runs["status"].str.lower().isin(["completed", "succeeded"])]
        failed      = runs[runs["status"].str.lower().isin(["failed", "cancelled"])]

        all_dur     = runs["duration_seconds"].dropna()
        ok_dur      = completed["duration_seconds"].dropna()
        fail_dur    = failed["duration_seconds"].dropna()

        total_sec   = all_dur.sum()
        ok_sec      = ok_dur.sum()
        wasted_sec  = fail_dur.sum()
        avg_dur_sec = all_dur.mean() if len(all_dur) > 0 else 0
        idle_est_sec = wasted_sec  # Wasted compute is analogous to idle time

        # Compute Efficiency
        compute_eff = (ok_sec / total_sec * 100) if total_sec > 0 else 0

        # Consistency = P50 / Mean (higher = runs are consistent in duration)
        consistency = 100.0
        if len(ok_dur) >= 2:
            p50  = ok_dur.median()
            mean = ok_dur.mean()
            consistency = min((p50 / mean * 100) if mean > 0 else 100, 100)

        # Estimated Spark Efficiency (weighted proxy)
        estimated_eff = compute_eff * 0.7 + consistency * 0.3

        results.append({
            "item_name":                nb["item_name"],
            "total_runs":               total_runs,
            "completed_runs":           len(completed),
            "failed_runs":              len(failed),
            "total_compute_min":        round(total_sec / 60, 1),
            "successful_compute_min":   round(ok_sec / 60, 1),
            "wasted_compute_min":       round(wasted_sec / 60, 1),
            "avg_duration_min":         round(avg_dur_sec / 60, 1),
            "idle_estimate_min":        round(idle_est_sec / 60, 1),
            "compute_efficiency_pct":   round(compute_eff, 1),
            "consistency_pct":          round(consistency, 1),
            "estimated_efficiency_pct": round(estimated_eff, 1),
        })

    return pd.DataFrame(results).sort_values("estimated_efficiency_pct") if results else pd.DataFrame()


# ═══════════════════════════════════════════════════════════════════════
# MAIN: Spark Resource Efficiency
# ═══════════════════════════════════════════════════════════════════════

print("⚡ SPARK RESOURCE EFFICIENCY")
print("=" * 130)

df_spark_api = pd.DataFrame()  # Holds real Spark data (if available)
df_spark_eff = pd.DataFrame()  # Holds estimated efficiency (always calculated)

if INCLUDE_NOTEBOOKS and notebooks:

    # ── Try Fabric Spark API for real core-utilization efficiency ──
    spark_apps, spark_source = try_get_spark_applications(WORKSPACE_ID, HEADERS, LOOKBACK_DAYS)

    if spark_apps:
        df_spark_api = parse_spark_app_efficiency(spark_apps)
        if not df_spark_api.empty:
            print(f"✅ Real Spark efficiency data retrieved via {spark_source}")
            print(f"   Applications found: {len(df_spark_api)}\n")

            eff_col = "spark_efficiency_pct"
            apps_with_eff = df_spark_api[df_spark_api[eff_col].notna()]
            apps_no_eff   = df_spark_api[df_spark_api[eff_col].isna()]

            if not apps_with_eff.empty:
                for _, row in apps_with_eff.iterrows():
                    eff = row[eff_col]
                    icon = "🟢" if eff >= 70 else ("🟡" if eff >= 50 else "🔴")
                    dur  = f"{row['total_duration_sec']/60:.1f} min" if row.get("total_duration_sec") else "N/A"
                    idle = f"{row['total_idle_sec']/60:.1f} min" if row.get("total_idle_sec") else "N/A"
                    print(f"   {icon} {row['notebook_name']}")
                    print(f"      Duration: {dur}  |  Idle: {idle}  |  Efficiency: {eff:.1f}%")

            if not apps_no_eff.empty:
                print(f"\n   ℹ️  {len(apps_no_eff)} app(s) without core-level efficiency data")

    else:
        print("ℹ️  Spark monitoring API not available via REST.")
        print("   📌 For real core-utilization efficiency, open individual notebook runs in Fabric portal:")
        print("      Notebook run → Resources tab → Spark resource usage")

    # ── Always compute estimated efficiency from job instances ─────
    if not df_runs.empty:
        df_spark_eff = compute_notebook_efficiency_from_jobs(df_runs, notebooks)

    if not df_spark_eff.empty:
        print(f"\n📊 NOTEBOOK COMPUTE EFFICIENCY DASHBOARD")
        print("=" * 140)
        header = (f"   {'Notebook':<40s} {'Runs':>5s} {'Pass':>5s} {'Fail':>5s} "
                  f"{'Total(min)':>10s} {'OK(min)':>8s} {'Wasted(min)':>11s} "
                  f"{'Compute%':>9s} {'Consist%':>9s} {'Effic%':>7s}")
        print(header)
        print("   " + "-" * 134)

        for _, row in df_spark_eff.iterrows():
            eff = row["estimated_efficiency_pct"]
            icon = "🟢" if eff >= 90 else ("🟡" if eff >= 70 else "🔴")
            print(f"   {icon} {row['item_name']:<38s} "
                  f"{row['total_runs']:>5d} {row['completed_runs']:>5d} {row['failed_runs']:>5d} "
                  f"{row['total_compute_min']:>10.1f} {row['successful_compute_min']:>8.1f} "
                  f"{row['wasted_compute_min']:>11.1f} "
                  f"{row['compute_efficiency_pct']:>8.1f}% {row['consistency_pct']:>8.1f}% "
                  f"{row['estimated_efficiency_pct']:>6.1f}%")

        avg_eff      = df_spark_eff["estimated_efficiency_pct"].mean()
        total_wasted = df_spark_eff["wasted_compute_min"].sum()
        total_compute = df_spark_eff["total_compute_min"].sum()
        print("   " + "-" * 134)
        print(f"   Average Efficiency: {avg_eff:.1f}%  |  "
              f"Total Compute: {total_compute:,.1f} min  |  Wasted: {total_wasted:,.1f} min")

        # ── Efficiency tier breakdown ────────────────────────────────
        high_eff = len(df_spark_eff[df_spark_eff["estimated_efficiency_pct"] >= 90])
        med_eff  = len(df_spark_eff[(df_spark_eff["estimated_efficiency_pct"] >= 70) &
                                    (df_spark_eff["estimated_efficiency_pct"] < 90)])
        low_eff  = len(df_spark_eff[df_spark_eff["estimated_efficiency_pct"] < 70])

        print(f"\n   🟢 Efficient (≥90%):   {high_eff} notebook(s)")
        print(f"   🟡 Moderate (70-89%):  {med_eff} notebook(s)")
        print(f"   🔴 Low (<70%):         {low_eff} notebook(s)")

        # ── Recommendations for low-efficiency notebooks ─────────────
        low_eff_items = df_spark_eff[df_spark_eff["estimated_efficiency_pct"] < 70]
        if not low_eff_items.empty:
            print(f"\n💡 LOW EFFICIENCY NOTEBOOKS – ACTION REQUIRED")
            print("-" * 80)
            for _, row in low_eff_items.iterrows():
                print(f"   🔴 '{row['item_name']}'  →  Efficiency: {row['estimated_efficiency_pct']:.1f}%")
                if row["failed_runs"] > 0:
                    print(f"      ⚠️  {row['failed_runs']} failed runs wasted {row['wasted_compute_min']:.1f} min of compute")
                if row["consistency_pct"] < 70:
                    print(f"      ⚠️  Duration variance is high (Consistency: {row['consistency_pct']:.1f}%) – "
                          "investigate outlier runs")

        # ── Metric definitions ───────────────────────────────────────
        print(f"\n📋 METRIC DEFINITIONS")
        print("-" * 80)
        print("   Compute %:     Successful compute time / Total compute time × 100")
        print("   Consistency %: Duration P50 / Mean × 100 (higher = runs are uniform)")
        print("   Efficiency %:  Weighted: 70% Compute + 30% Consistency")
        print()
        print("   💡 For real Spark core-utilization efficiency (executor idle time),")
        print("      open individual notebook runs → Resources tab → Spark resource usage")
    else:
        print("   ℹ️  No notebook run data available for efficiency calculation.")

else:
    print("   ⏭️  Skipped – notebook tracking is disabled or no notebooks found.")

## 🔟 Consolidated Summary

High-level summary comparing Notebooks vs Pipelines efficiency across all metrics.

In [ ]:
# ── Consolidated Summary ─────────────────────────────────────────────

if not df_efficiency.empty:
    print("📊 CONSOLIDATED EFFICIENCY SUMMARY")
    print("=" * 120)

    for itype in ["Notebook", "DataPipeline"]:
        subset = df_efficiency[df_efficiency["item_type"] == itype]
        if subset.empty:
            continue

        icon = "📓" if itype == "Notebook" else "🔄"
        label = "Notebooks" if itype == "Notebook" else "Data Pipelines"

        total_items = len(subset)
        total_runs = subset["total_runs"].sum()
        total_success = subset["success_count"].sum()
        total_failures = subset["failure_count"].sum()
        overall_rate = (total_success / total_runs * 100) if total_runs > 0 else 0
        total_compute = subset["total_compute_hrs"].sum()
        avg_duration_all = subset["avg_duration_min"].mean() if subset["avg_duration_min"].notna().any() else 0

        print(f"\n{icon} {label}")
        print("-" * 80)
        print(f"   Items:              {total_items}")
        print(f"   Total Runs:         {total_runs:,}")
        print(f"   Successful:         {total_success:,}")
        print(f"   Failed/Cancelled:   {total_failures:,}")
        print(f"   Success Rate:       {overall_rate:.1f}%")
        print(f"   Avg Duration:       {avg_duration_all:.1f} min")
        print(f"   Total Compute:      {total_compute:,.1f} hours")

        # ── Spark Efficiency summary (Notebooks only) ────────────
        if itype == "Notebook" and 'df_spark_eff' in dir() and not df_spark_eff.empty:
            avg_eff      = df_spark_eff["estimated_efficiency_pct"].mean()
            wasted_min   = df_spark_eff["wasted_compute_min"].sum()
            low_count    = len(df_spark_eff[df_spark_eff["estimated_efficiency_pct"] < 70])
            print(f"   ⚡ Avg Spark Efficiency: {avg_eff:.1f}%")
            print(f"   ⚡ Wasted Compute:       {wasted_min:,.1f} min")
            if low_count > 0:
                print(f"   ⚡ Low Efficiency (<70%): {low_count} notebook(s)")

        # Top 3 longest running
        top3 = subset.nlargest(3, "avg_duration_min")
        if not top3.empty:
            print(f"   🐢 Slowest Items:")
            for _, row in top3.iterrows():
                print(f"      - {row['item_name']}: avg {row['avg_duration_min']:.1f} min")

        # Top 3 most failing
        top3_fail = subset[subset["failure_count"] > 0].nlargest(3, "failure_count")
        if not top3_fail.empty:
            print(f"   ❌ Most Failing:")
            for _, row in top3_fail.iterrows():
                print(f"      - {row['item_name']}: {row['failure_count']} failures ({row['success_rate_pct']}% success)")

        # Top 3 lowest efficiency (Notebooks only)
        if itype == "Notebook" and 'df_spark_eff' in dir() and not df_spark_eff.empty:
            low3 = df_spark_eff.nsmallest(3, "estimated_efficiency_pct")
            if not low3.empty:
                print(f"   ⚡ Least Efficient:")
                for _, row in low3.iterrows():
                    print(f"      - {row['item_name']}: {row['estimated_efficiency_pct']:.1f}% efficiency "
                          f"({row['wasted_compute_min']:.1f} min wasted)")

    # ── Recommendations ──────────────────────────────────────────────
    print("\n\n💡 RECOMMENDATIONS")
    print("=" * 80)

    recs = []

    # Items with low success rate
    low_success = df_efficiency[df_efficiency["success_rate_pct"] < 80]
    if not low_success.empty:
        for _, row in low_success.iterrows():
            recs.append(f"🔴 '{row['item_name']}' has {row['success_rate_pct']}% success rate – investigate failures")

    # Items with degrading trend
    degrading = df_efficiency[df_efficiency["duration_trend"].str.contains("Degrading", na=False)]
    if not degrading.empty:
        for _, row in degrading.iterrows():
            recs.append(f"⬇️ '{row['item_name']}' shows performance degradation – review recent changes")

    # Items with very long P95
    if df_efficiency["p95_duration_min"].notna().any():
        long_p95 = df_efficiency[df_efficiency["p95_duration_min"] > df_efficiency["avg_duration_min"] * 3]
        long_p95 = long_p95[long_p95["p95_duration_min"].notna()]
        for _, row in long_p95.iterrows():
            recs.append(f"📊 '{row['item_name']}' has high P95 variance (avg: {row['avg_duration_min']:.1f} min, "
                       f"P95: {row['p95_duration_min']:.1f} min) – consider investigating outlier runs")

    # Low Spark efficiency notebooks
    if 'df_spark_eff' in dir() and not df_spark_eff.empty:
        low_eff_items = df_spark_eff[df_spark_eff["estimated_efficiency_pct"] < 70]
        for _, row in low_eff_items.iterrows():
            recs.append(f"⚡ '{row['item_name']}' has low compute efficiency ({row['estimated_efficiency_pct']:.1f}%) – "
                       f"{row['wasted_compute_min']:.1f} min wasted on failed runs")

    if recs:
        for i, rec in enumerate(recs, 1):
            print(f"   {i}. {rec}")
    else:
        print("   ✅ All items are performing well. No immediate action required.")
else:
    print("⚠️  No data to summarize.")

## 1️⃣1️⃣ Export Report to CSV

Saves the full efficiency report and detailed run history to CSV files for further analysis.

In [ ]:
# ── Export to CSV ───────────────────────────────────────────────────

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# ── Export efficiency summary ────────────────────────────────────────
if not df_efficiency.empty:
    summary_file = f"fabric_efficiency_summary_{timestamp}.csv"
    export_cols = [
        "item_type", "item_name", "total_runs", "success_count", "failure_count",
        "success_rate_pct", "avg_duration_min", "p50_duration_min", "p95_duration_min",
        "min_duration_min", "max_duration_min", "total_compute_hrs",
        "last_run_time", "last_status", "duration_trend", "top_failure_reason"
    ]
    df_efficiency[[c for c in export_cols if c in df_efficiency.columns]].to_csv(
        summary_file, index=False
    )
    print(f"✅ Efficiency summary exported to: {summary_file}")
    print(f"   Rows: {len(df_efficiency)}")

# ── Export detailed run history ──────────────────────────────────────
if not df_runs.empty:
    runs_file = f"fabric_run_history_{timestamp}.csv"
    df_runs_export = df_runs.copy()
    # Convert datetime for CSV compatibility
    for col in ["start_time", "end_time"]:
        if col in df_runs_export.columns:
            df_runs_export[col] = df_runs_export[col].astype(str)
    df_runs_export.to_csv(runs_file, index=False)
    print(f"✅ Run history exported to: {runs_file}")
    print(f"   Rows: {len(df_runs_export)}")

# ── Export Spark efficiency (Notebooks) ──────────────────────────────
if 'df_spark_eff' in dir() and not df_spark_eff.empty:
    spark_file = f"fabric_spark_efficiency_{timestamp}.csv"
    df_spark_eff.to_csv(spark_file, index=False)
    print(f"✅ Spark efficiency exported to: {spark_file}")
    print(f"   Rows: {len(df_spark_eff)}")

# ── Export real Spark API data (if available) ────────────────────────
if 'df_spark_api' in dir() and not df_spark_api.empty:
    spark_api_file = f"fabric_spark_api_data_{timestamp}.csv"
    df_spark_api.to_csv(spark_api_file, index=False)
    print(f"✅ Spark API data exported to: {spark_api_file}")
    print(f"   Rows: {len(df_spark_api)}")

if df_efficiency.empty and df_runs.empty:
    print("⚠️  No data to export.")

---

## 📝 Notes

### Running This Notebook
- **Import** into any Fabric workspace via the Fabric portal
- **Attach a Lakehouse** (optional) to persist CSV exports as files
- Set `TARGET_WORKSPACE_ID` to scan a different workspace, or leave `None` for the current one
- Adjust `LOOKBACK_DAYS` to control the analysis window (default: 30 days)

### Fabric REST API Endpoints Used

| Endpoint | Purpose |
|----------|---------|
| `GET /v1/workspaces/{id}` | Workspace metadata |
| `GET /v1/workspaces/{id}/items?type=Notebook` | List all notebooks |
| `GET /v1/workspaces/{id}/items?type=DataPipeline` | List all data pipelines |
| `GET /v1/workspaces/{id}/items/{id}/jobs/instances` | Run history per item |
| `GET /v1/workspaces/{id}/spark/applications` | Spark application metrics (if available) |
| `GET /v1/workspaces/{id}/spark/livySessions` | Livy session data (fallback) |

### Spark Resource Efficiency

The **Spark Resource Efficiency %** shown in the Fabric portal (Resources tab) measures how effectively
Spark executor cores are utilized:

```
Efficiency = Active Executor-Core-Seconds / Allocated Executor-Core-Seconds × 100
```

This notebook attempts to retrieve this real efficiency via the Fabric Spark API. If the API is not
accessible, it calculates a **Compute Efficiency** proxy:

| Metric | Formula | What It Measures |
|--------|---------|------------------|
| **Compute %** | `Successful_Time / Total_Time × 100` | How much compute was productive vs. wasted on failures |
| **Consistency %** | `Duration_P50 / Duration_Mean × 100` | How uniform run durations are (higher = less variance) |
| **Efficiency %** | `70% × Compute + 30% × Consistency` | Combined proxy for overall Spark efficiency |

To view the **real** Spark core-utilization efficiency per run:
1. Open a notebook run in the Fabric portal
2. Click the **Resources** tab
3. View **Spark resource usage** → Total duration / Idle time / Efficiency %

### Metrics Explained

| Metric | Description |
|--------|-------------|
| **Success Rate** | `successful_runs / total_runs × 100` |
| **Avg Duration** | Mean execution time across all completed runs |
| **P50 (Median)** | 50th percentile – half the runs are faster |
| **P95** | 95th percentile – 95% of runs are faster |
| **Total Compute** | Cumulative hours of execution time consumed |
| **Trend** | Compares avg duration of recent runs vs older runs |
| **Wasted Compute** | Time consumed by failed/cancelled runs |

### Health Thresholds

| Status | Criteria | Action |
|--------|----------|--------|
| 🟢 Healthy | ≥ 95% success rate | No action needed |
| 🟡 Warning | 80–94% success rate | Monitor and investigate |
| 🔴 Critical | < 80% success rate | Immediate investigation required |

### Spark Efficiency Thresholds

| Status | Criteria | Action |
|--------|----------|--------|
| 🟢 Efficient | ≥ 90% efficiency | Well-optimized |
| 🟡 Moderate | 70–89% efficiency | Review for optimization opportunities |
| 🔴 Low | < 70% efficiency | Investigate failures, duration variance, and Spark resource usage |

### Permissions Required
- **Workspace Admin** or **Member** role for full access to job instances
- **Viewer** role may have limited access to run history

### Limitations
- Run history retention depends on your Fabric capacity tier
- Real Spark core-utilization efficiency requires the Spark Applications API (availability may vary)
- CU (Capacity Unit) consumption details require Admin API access
- Very long-running pipelines with many activities may have truncated details